# 给用户推荐标签

In [1]:
# 导入包
import random
import math
import time
from tqdm import tqdm

## 一. 通用函数定义

In [2]:
# 定义装饰器，监控运行时间
def timmer(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        res = func(*args, **kwargs)
        stop_time = time.time()
        print('Func %s, run time: %s' % (func.__name__, stop_time - start_time))
        return res
    return wrapper

### 1. 数据处理相关
Delicious-2k数据集
1. load data
2. split data

In [15]:
class Dataset():
    
    def __init__(self, fp):
        # fp: data file path
        self.data = self.loadData(fp)
    
    @timmer
    def loadData(self, fp):
        data = [f.strip().split('\t')[:3] for f in open(fp).readlines()[1:]]
        return data
    
    @timmer
    def splitData(self, M, k, seed=1):
        '''
        :params: data, 加载的所有(user, item)数据条目
        :params: M, 划分的数目，最后需要取M折的平均
        :params: k, 本次是第几次划分，k~[0, M)
        :params: seed, random的种子数，对于不同的k应设置成一样的
        :return: train, test
        '''
        # 按照(user, item)作为key进行划分
        train, test = [], []
        random.seed(seed)
        for user, item, tag in self.data:
            # 这里与书中的不一致，本人认为取M-1较为合理，因randint是左右都覆盖的
            if random.randint(0, M-1) == k:  
                test.append((user, item, tag))
            else:
                train.append((user, item, tag))

        # 处理成字典的形式，user->set(items)
        def convert_dict(data):
            data_dict = {}
            for user, item, tag in data:
                if user not in data_dict:
                    data_dict[user] = {}
                if item not in data_dict[user]:
                    data_dict[user][item] = set()
                data_dict[user][item].add(tag)
            for user in data_dict:
                for item in data_dict[user]:
                    data_dict[user][item] = list(data_dict[user][item])
            return data_dict

        return convert_dict(train), convert_dict(test)

### 2. 评价指标
1. Precision
2. Recall
3. Coverage
4. Diversity
5. Popularity(Novelty)

In [4]:
class Metric():
    
    def __init__(self, train, test, GetRecommendation):
        '''
        :params: train, 训练数据
        :params: test, 测试数据
        :params: GetRecommendation, 为某个用户获取推荐物品的接口函数
        '''
        self.train = train
        self.test = test
        self.GetRecommendation = GetRecommendation
        self.recs = self.getRec()
        
    # 为test中的每个用户进行推荐
    def getRec(self):
        recs = {}
        for user in self.test:
            recs[user] = {}
            for item in self.test[user]:
                rank = self.GetRecommendation(user, item)
                recs[user][item] = rank
        return recs
        
    # 定义精确率指标计算方式
    def precision(self):
        all, hit = 0, 0
        for user in self.test:
            for item in self.test[user]:
                test_tags = set(self.test[user][item])
                rank = self.recs[user][item]
                for tag, score in rank:
                    if tag in test_tags:
                        hit += 1
                all += len(rank)
        return round(hit / all * 100, 2)
    
    # 定义召回率指标计算方式
    def recall(self):
        all, hit = 0, 0
        for user in self.test:
            for item in self.test[user]:
                test_tags = set(self.test[user][item])
                rank = self.recs[user][item]
                for tag, score in rank:
                    if tag in test_tags:
                        hit += 1
                all += len(test_tags)
        return round(hit / all * 100, 2)
    
    def eval(self):
        metric = {'Precision': self.precision(),
                  'Recall': self.recall()}
        print('Metric:', metric)
        return metric

## 二. 算法实现
1. Popular
2. UserPopular
3. ItemPopular
4. HybridPopular

In [17]:
# 1. 推荐热门标签
def Popular(train, N):
    '''
    :params: train, 训练数据集
    :params: N, 超参数，设置取TopN推荐物品数目
    :return: GetRecommendation，推荐接口函数
    '''
    # 统计tags
    tags = {}
    for user in train:
        for item in train[user]:
            for tag in train[user][item]:
                if tag not in tags:
                    tags[tag] = 0
                tags[tag] += 1
    tags = list(sorted(tags.items(), key=lambda x: x[1], reverse=True))[:N]
    
    def GetRecommendation(user, item):
        return tags
    
    return GetRecommendation

In [6]:
# 2. 推荐用户最热门的标签
def UserPopular(train, N):
    '''
    :params: train, 训练数据集
    :params: N, 超参数，设置取TopN推荐物品数目
    :return: GetRecommendation，推荐接口函数
    '''
    # 统计user_tags
    user_tags = {}
    for user in train:
        user_tags[user] = {}
        for item in train[user]:
            for tag in train[user][item]:
                if tag not in user_tags[user]:
                    user_tags[user][tag] = 0
                user_tags[user][tag] += 1
    user_tags = {k: list(sorted(v.items(), key=lambda x: x[1], reverse=True)) 
                 for k, v in user_tags.items()}
    
    def GetRecommendation(user, item):
        return user_tags[user][:N] if user in user_tags else []
    
    return GetRecommendation

In [7]:
# 3. 推荐物品最热门的标签
def ItemPopular(train, N):
    '''
    :params: train, 训练数据集
    :params: N, 超参数，设置取TopN推荐物品数目
    :return: GetRecommendation，推荐接口函数
    '''
    # 统计item_tags
    item_tags = {}
    for user in train:
        for item in train[user]:
            if item not in item_tags:
                item_tags[item] = {}
            for tag in train[user][item]:
                if tag not in item_tags[item]:
                    item_tags[item][tag] = 0
                item_tags[item][tag] += 1
    item_tags = {k: list(sorted(v.items(), key=lambda x: x[1], reverse=True)) 
                 for k, v in item_tags.items()}
    
    def GetRecommendation(user, item):
        return item_tags[item][:N] if item in item_tags else []
    
    return GetRecommendation

In [31]:
# 4. 联合用户和物品进行推荐
def HybridPopular(train, N, alpha):
    '''
    :params: train, 训练数据集
    :params: N, 超参数，设置取TopN推荐物品数目
    :params: alpha，超参数，设置用户和物品的融合比例
    :return: GetRecommendation，推荐接口函数
    '''

    # 统计user_tags
    user_tags = {}
    for user in train:
        user_tags[user] = {}
        for item in train[user]:
            for tag in train[user][item]:
                if tag not in user_tags[user]:
                    user_tags[user][tag] = 0
                user_tags[user][tag] += 1
                
    # 统计item_tags
    item_tags = {}
    for user in train:
        for item in train[user]:
            if item not in item_tags:
                item_tags[item] = {}
            for tag in train[user][item]:
                if tag not in item_tags[item]:
                    item_tags[item][tag] = 0
                item_tags[item][tag] += 1
    
    def GetRecommendation(user, item):
        tag_score = {}
        if user in user_tags:
            max_user_tag = max(user_tags[user].values())
            for tag in user_tags[user]:
                if tag not in tag_score:
                    tag_score[tag] = 0
                tag_score[tag] += (1 - alpha) * user_tags[user][tag] / max_user_tag
        if item in item_tags:
            max_item_tag = max(item_tags[item].values())
            for tag in item_tags[item]:
                if tag not in tag_score:
                    tag_score[tag] = 0
                tag_score[tag] += alpha * item_tags[item][tag] / max_item_tag
        return list(sorted(tag_score.items(), key=lambda x: x[1], reverse=True))[:N]
    
    return GetRecommendation

## 三. 实验
1. Popular实验
2. UserPopular实验
3. ItemPopular实验
4. HybridPopular实验

M=10, N=10

In [24]:
class Experiment():
    
    def __init__(self, M, N, fp='../dataset/delicious-2k/user_taggedbookmarks.dat', rt='Popular'):
        '''
        :params: M, 进行多少次实验
        :params: N, TopN推荐物品的个数
        :params: fp, 数据文件路径
        :params: rt, 推荐算法类型
        '''
        self.M = M
        self.N = N
        self.fp = fp
        self.rt = rt
        self.alg = {'Popular': Popular, 'UserPopular': UserPopular, \
                    'ItemPopular': ItemPopular, 'HybridPopular': HybridPopular}
    
    # 定义单次实验
    @timmer
    def worker(self, train, test, **kwargs):
        '''
        :params: train, 训练数据集
        :params: test, 测试数据集
        :return: 各指标的值
        '''
        getRecommendation = self.alg[self.rt](train, self.N, **kwargs)
        metric = Metric(train, test, getRecommendation)
        return metric.eval()
    
    # 多次实验取平均
    @timmer
    def run(self, **kwargs):
        metrics = {'Precision': 0, 'Recall': 0}
        dataset = Dataset(self.fp)
        for ii in range(self.M):
            train, test = dataset.splitData(self.M, ii)
            print('Experiment {}:'.format(ii))
            metric = self.worker(train, test, **kwargs)
            metrics = {k: metrics[k]+metric[k] for k in metrics}
        metrics = {k: metrics[k] / self.M for k in metrics}
        print('Average Result (M={}, N={}): {}'.format(\
                              self.M, self.N, metrics))

In [20]:
# 1. Popular实验
M, N = 10, 10
exp = Experiment(M, N, rt='Popular')
exp.run()

Func loadData, run time: 0.7870762348175049
Func splitData, run time: 1.234386920928955
Experiment 0:
Metric: {'Precision': 0.87, 'Recall': 6.79}
Func worker, run time: 0.26900410652160645
Func splitData, run time: 1.0280041694641113
Experiment 1:
Metric: {'Precision': 0.83, 'Recall': 6.52}
Func worker, run time: 0.19060897827148438
Func splitData, run time: 1.0345518589019775
Experiment 2:
Metric: {'Precision': 0.84, 'Recall': 6.52}
Func worker, run time: 0.20136189460754395
Func splitData, run time: 1.0786409378051758
Experiment 3:
Metric: {'Precision': 0.84, 'Recall': 6.6}
Func worker, run time: 0.19781804084777832
Func splitData, run time: 1.1649599075317383
Experiment 4:
Metric: {'Precision': 0.85, 'Recall': 6.7}
Func worker, run time: 0.19120001792907715
Func splitData, run time: 1.0572938919067383
Experiment 5:
Metric: {'Precision': 0.83, 'Recall': 6.52}
Func worker, run time: 0.20640897750854492
Func splitData, run time: 0.8927929401397705
Experiment 6:
Metric: {'Precision': 0.

In [21]:
# 2. UserPopular实验
M, N = 10, 10
exp = Experiment(M, N, rt='UserPopular')
exp.run()

Func loadData, run time: 0.6694128513336182
Func splitData, run time: 1.1229050159454346
Experiment 0:
Metric: {'Precision': 3.2, 'Recall': 24.96}
Func worker, run time: 0.29575133323669434
Func splitData, run time: 1.0629141330718994
Experiment 1:
Metric: {'Precision': 3.19, 'Recall': 24.88}
Func worker, run time: 0.3048582077026367
Func splitData, run time: 1.0542120933532715
Experiment 2:
Metric: {'Precision': 3.18, 'Recall': 24.69}
Func worker, run time: 0.29949402809143066
Func splitData, run time: 0.9658150672912598
Experiment 3:
Metric: {'Precision': 3.19, 'Recall': 24.84}
Func worker, run time: 0.4243886470794678
Func splitData, run time: 0.962958812713623
Experiment 4:
Metric: {'Precision': 3.17, 'Recall': 24.79}
Func worker, run time: 0.4475588798522949
Func splitData, run time: 0.9121999740600586
Experiment 5:
Metric: {'Precision': 3.12, 'Recall': 24.33}
Func worker, run time: 0.46161675453186035
Func splitData, run time: 0.9740099906921387
Experiment 6:
Metric: {'Precision'

In [22]:
# 3. ItemPopular实验
M, N = 10, 10
exp = Experiment(M, N, rt='ItemPopular')
exp.run()

Func loadData, run time: 0.555941104888916
Func splitData, run time: 1.1046972274780273
Experiment 0:
Metric: {'Precision': 1.67, 'Recall': 8.19}
Func worker, run time: 0.7694768905639648
Func splitData, run time: 0.9527928829193115
Experiment 1:
Metric: {'Precision': 1.57, 'Recall': 7.66}
Func worker, run time: 0.7398290634155273
Func splitData, run time: 0.9173603057861328
Experiment 2:
Metric: {'Precision': 1.62, 'Recall': 7.9}
Func worker, run time: 0.655925989151001
Func splitData, run time: 1.093824863433838
Experiment 3:
Metric: {'Precision': 1.6, 'Recall': 7.77}
Func worker, run time: 0.49861979484558105
Func splitData, run time: 1.1611483097076416
Experiment 4:
Metric: {'Precision': 1.63, 'Recall': 7.97}
Func worker, run time: 0.5834429264068604
Func splitData, run time: 1.0758311748504639
Experiment 5:
Metric: {'Precision': 1.62, 'Recall': 7.95}
Func worker, run time: 0.5447440147399902
Func splitData, run time: 1.1658220291137695
Experiment 6:
Metric: {'Precision': 1.62, 'Re

In [33]:
# 4. HybridPopular实验
M, N = 10, 10
for alpha in range(0, 11):
    alpha /= 10
    print('alpha =', alpha)
    exp = Experiment(M, N, rt='HybridPopular')
    exp.run(alpha=alpha)

alpha = 0.0
Func loadData, run time: 0.8270349502563477
Func splitData, run time: 1.1357169151306152
Experiment 0:
Metric: {'Precision': 3.2, 'Recall': 24.98}
Func worker, run time: 3.6687381267547607
Func splitData, run time: 1.313784122467041
Experiment 1:
Metric: {'Precision': 3.19, 'Recall': 24.89}
Func worker, run time: 3.9108307361602783
Func splitData, run time: 1.080482006072998
Experiment 2:
Metric: {'Precision': 3.17, 'Recall': 24.7}
Func worker, run time: 3.9596028327941895
Func splitData, run time: 1.113774061203003
Experiment 3:
Metric: {'Precision': 3.19, 'Recall': 24.86}
Func worker, run time: 3.9999098777770996
Func splitData, run time: 1.1503219604492188
Experiment 4:
Metric: {'Precision': 3.17, 'Recall': 24.8}
Func worker, run time: 3.866776943206787
Func splitData, run time: 0.9508001804351807
Experiment 5:
Metric: {'Precision': 3.12, 'Recall': 24.35}
Func worker, run time: 4.116667985916138
Func splitData, run time: 0.9126389026641846
Experiment 6:
Metric: {'Precisi

Metric: {'Precision': 2.83, 'Recall': 22.12}
Func worker, run time: 3.846303939819336
Func splitData, run time: 1.1365282535552979
Experiment 1:
Metric: {'Precision': 2.82, 'Recall': 21.99}
Func worker, run time: 3.859900951385498
Func splitData, run time: 0.9495680332183838
Experiment 2:
Metric: {'Precision': 2.83, 'Recall': 22.04}
Func worker, run time: 4.079281806945801
Func splitData, run time: 0.9861047267913818
Experiment 3:
Metric: {'Precision': 2.83, 'Recall': 22.06}
Func worker, run time: 4.394114017486572
Func splitData, run time: 1.0009009838104248
Experiment 4:
Metric: {'Precision': 2.82, 'Recall': 22.1}
Func worker, run time: 4.027346134185791
Func splitData, run time: 0.8993139266967773
Experiment 5:
Metric: {'Precision': 2.81, 'Recall': 21.98}
Func worker, run time: 4.053209066390991
Func splitData, run time: 0.9647109508514404
Experiment 6:
Metric: {'Precision': 2.83, 'Recall': 22.13}
Func worker, run time: 4.376596927642822
Func splitData, run time: 1.0014677047729492


Metric: {'Precision': 1.8, 'Recall': 14.09}
Func worker, run time: 3.4529192447662354
Func splitData, run time: 1.1971077919006348
Experiment 2:
Metric: {'Precision': 1.82, 'Recall': 14.17}
Func worker, run time: 3.3798270225524902
Func splitData, run time: 0.9837942123413086
Experiment 3:
Metric: {'Precision': 1.79, 'Recall': 13.97}
Func worker, run time: 3.9176063537597656
Func splitData, run time: 0.9464359283447266
Experiment 4:
Metric: {'Precision': 1.8, 'Recall': 14.13}
Func worker, run time: 3.7294299602508545
Func splitData, run time: 0.9589240550994873
Experiment 5:
Metric: {'Precision': 1.81, 'Recall': 14.16}
Func worker, run time: 3.68717098236084
Func splitData, run time: 0.9121508598327637
Experiment 6:
Metric: {'Precision': 1.82, 'Recall': 14.22}
Func worker, run time: 3.720194101333618
Func splitData, run time: 0.9417569637298584
Experiment 7:
Metric: {'Precision': 1.8, 'Recall': 14.05}
Func worker, run time: 3.7353739738464355
Func splitData, run time: 0.964847087860107

## 四. 实验结果
1. Popular实验

    Running time: 13.749655961990356
    
    Average Result (M=10, N=10): {'Precision': 0.844, 'Recall': 6.607000000000001}
Func run, run time: 13.749655961990356
     
2. UserPopular实验
    
    Running time: 14.44516921043396
    
    Average Result (M=10, N=10): {'Precision': 3.1689999999999996, 'Recall': 24.695000000000004}
     
3. ItemPopular实验
    
    Running time: 17.594825983047485
    
    Average Result (M=10, N=10): {'Precision': 1.614, 'Recall': 7.881}

4. HybridPopular实验（因为此数据集里面的ItemPopular效果特别不好，所以随着alpha增大，效果越来越差）

    alpha = 0.0
    Average Result (M=10, N=10): {'Precision': 3.1659999999999995, 'Recall': 24.71}
    
    alpha = 0.1
    Average Result (M=10, N=10): {'Precision': 3.135, 'Recall': 24.48}
    
    alpha = 0.2
    Average Result (M=10, N=10): {'Precision': 3.0599999999999996, 'Recall': 23.889999999999997}
    
    alpha = 0.3
    Average Result (M=10, N=10): {'Precision': 2.9669999999999996, 'Recall': 23.154000000000003}
    
    alpha = 0.4
    Average Result (M=10, N=10): {'Precision': 2.887, 'Recall': 22.540999999999997}
    
    alpha = 0.5
    Average Result (M=10, N=10): {'Precision': 2.8180000000000005, 'Recall': 21.998}
    
    alpha = 0.6
    Average Result (M=10, N=10): {'Precision': 2.803, 'Recall': 21.891000000000002}
    
    alpha = 0.7
    Average Result (M=10, N=10): {'Precision': 2.791, 'Recall': 21.784000000000002}
    
    alpha = 0.8
    Average Result (M=10, N=10): {'Precision': 2.78, 'Recall': 21.707}
    
    alpha = 0.9
    Average Result (M=10, N=10): {'Precision': 2.774, 'Recall': 21.657}
    
    alpha = 1.0
    Average Result (M=10, N=10): {'Precision': 1.8050000000000002, 'Recall': 14.094999999999999}

## 五. 总结
1. 运行结果与书中的结果很多都不相符，怀疑是数据集的原因，书中并没有提供数据集，之后还是要根据实际情况来调

## 附：运行日志（请双击看）

1. Popular实验
Func loadData, run time: 0.7870762348175049
Func splitData, run time: 1.234386920928955
Experiment 0:
Metric: {'Precision': 0.87, 'Recall': 6.79}
Func worker, run time: 0.26900410652160645
Func splitData, run time: 1.0280041694641113
Experiment 1:
Metric: {'Precision': 0.83, 'Recall': 6.52}
Func worker, run time: 0.19060897827148438
Func splitData, run time: 1.0345518589019775
Experiment 2:
Metric: {'Precision': 0.84, 'Recall': 6.52}
Func worker, run time: 0.20136189460754395
Func splitData, run time: 1.0786409378051758
Experiment 3:
Metric: {'Precision': 0.84, 'Recall': 6.6}
Func worker, run time: 0.19781804084777832
Func splitData, run time: 1.1649599075317383
Experiment 4:
Metric: {'Precision': 0.85, 'Recall': 6.7}
Func worker, run time: 0.19120001792907715
Func splitData, run time: 1.0572938919067383
Experiment 5:
Metric: {'Precision': 0.83, 'Recall': 6.52}
Func worker, run time: 0.20640897750854492
Func splitData, run time: 0.8927929401397705
Experiment 6:
Metric: {'Precision': 0.88, 'Recall': 6.88}
Func worker, run time: 0.18125414848327637
Func splitData, run time: 1.024669885635376
Experiment 7:
Metric: {'Precision': 0.8, 'Recall': 6.27}
Func worker, run time: 0.18497800827026367
Func splitData, run time: 1.035869836807251
Experiment 8:
Metric: {'Precision': 0.84, 'Recall': 6.57}
Func worker, run time: 0.20566082000732422
Func splitData, run time: 1.166532039642334
Experiment 9:
Metric: {'Precision': 0.86, 'Recall': 6.7}
Func worker, run time: 0.22189784049987793
Average Result (M=10, N=10): {'Precision': 0.844, 'Recall': 6.607000000000001}
Func run, run time: 13.749655961990356

2. UserPopular实验
Func loadData, run time: 0.6694128513336182
Func splitData, run time: 1.1229050159454346
Experiment 0:
Metric: {'Precision': 3.2, 'Recall': 24.96}
Func worker, run time: 0.29575133323669434
Func splitData, run time: 1.0629141330718994
Experiment 1:
Metric: {'Precision': 3.19, 'Recall': 24.88}
Func worker, run time: 0.3048582077026367
Func splitData, run time: 1.0542120933532715
Experiment 2:
Metric: {'Precision': 3.18, 'Recall': 24.69}
Func worker, run time: 0.29949402809143066
Func splitData, run time: 0.9658150672912598
Experiment 3:
Metric: {'Precision': 3.19, 'Recall': 24.84}
Func worker, run time: 0.4243886470794678
Func splitData, run time: 0.962958812713623
Experiment 4:
Metric: {'Precision': 3.17, 'Recall': 24.79}
Func worker, run time: 0.4475588798522949
Func splitData, run time: 0.9121999740600586
Experiment 5:
Metric: {'Precision': 3.12, 'Recall': 24.33}
Func worker, run time: 0.46161675453186035
Func splitData, run time: 0.9740099906921387
Experiment 6:
Metric: {'Precision': 3.17, 'Recall': 24.74}
Func worker, run time: 0.4318516254425049
Func splitData, run time: 0.8616311550140381
Experiment 7:
Metric: {'Precision': 3.13, 'Recall': 24.43}
Func worker, run time: 0.4322950839996338
Func splitData, run time: 0.8580801486968994
Experiment 8:
Metric: {'Precision': 3.17, 'Recall': 24.65}
Func worker, run time: 0.42772817611694336
Func splitData, run time: 0.8563971519470215
Experiment 9:
Metric: {'Precision': 3.17, 'Recall': 24.64}
Func worker, run time: 0.43137383460998535
Average Result (M=10, N=10): {'Precision': 3.1689999999999996, 'Recall': 24.695000000000004}
Func run, run time: 14.44516921043396

3. ItemPopular实验
Func loadData, run time: 0.555941104888916
Func splitData, run time: 1.1046972274780273
Experiment 0:
Metric: {'Precision': 1.67, 'Recall': 8.19}
Func worker, run time: 0.7694768905639648
Func splitData, run time: 0.9527928829193115
Experiment 1:
Metric: {'Precision': 1.57, 'Recall': 7.66}
Func worker, run time: 0.7398290634155273
Func splitData, run time: 0.9173603057861328
Experiment 2:
Metric: {'Precision': 1.62, 'Recall': 7.9}
Func worker, run time: 0.655925989151001
Func splitData, run time: 1.093824863433838
Experiment 3:
Metric: {'Precision': 1.6, 'Recall': 7.77}
Func worker, run time: 0.49861979484558105
Func splitData, run time: 1.1611483097076416
Experiment 4:
Metric: {'Precision': 1.63, 'Recall': 7.97}
Func worker, run time: 0.5834429264068604
Func splitData, run time: 1.0758311748504639
Experiment 5:
Metric: {'Precision': 1.62, 'Recall': 7.95}
Func worker, run time: 0.5447440147399902
Func splitData, run time: 1.1658220291137695
Experiment 6:
Metric: {'Precision': 1.62, 'Recall': 7.92}
Func worker, run time: 0.570044994354248
Func splitData, run time: 1.09977388381958
Experiment 7:
Metric: {'Precision': 1.61, 'Recall': 7.88}
Func worker, run time: 0.5018339157104492
Func splitData, run time: 1.0852570533752441
Experiment 8:
Metric: {'Precision': 1.61, 'Recall': 7.83}
Func worker, run time: 0.5137588977813721
Func splitData, run time: 1.1302859783172607
Experiment 9:
Metric: {'Precision': 1.59, 'Recall': 7.74}
Func worker, run time: 0.6344318389892578
Average Result (M=10, N=10): {'Precision': 1.614, 'Recall': 7.881}
Func run, run time: 17.594825983047485

4. HybridPopular实验
alpha = 0.0
Func loadData, run time: 0.8270349502563477
Func splitData, run time: 1.1357169151306152
Experiment 0:
Metric: {'Precision': 3.2, 'Recall': 24.98}
Func worker, run time: 3.6687381267547607
Func splitData, run time: 1.313784122467041
Experiment 1:
Metric: {'Precision': 3.19, 'Recall': 24.89}
Func worker, run time: 3.9108307361602783
Func splitData, run time: 1.080482006072998
Experiment 2:
Metric: {'Precision': 3.17, 'Recall': 24.7}
Func worker, run time: 3.9596028327941895
Func splitData, run time: 1.113774061203003
Experiment 3:
Metric: {'Precision': 3.19, 'Recall': 24.86}
Func worker, run time: 3.9999098777770996
Func splitData, run time: 1.1503219604492188
Experiment 4:
Metric: {'Precision': 3.17, 'Recall': 24.8}
Func worker, run time: 3.866776943206787
Func splitData, run time: 0.9508001804351807
Experiment 5:
Metric: {'Precision': 3.12, 'Recall': 24.35}
Func worker, run time: 4.116667985916138
Func splitData, run time: 0.9126389026641846
Experiment 6:
Metric: {'Precision': 3.16, 'Recall': 24.75}
Func worker, run time: 4.2959370613098145
Func splitData, run time: 0.9400320053100586
Experiment 7:
Metric: {'Precision': 3.13, 'Recall': 24.45}
Func worker, run time: 4.016885042190552
Func splitData, run time: 0.9569549560546875
Experiment 8:
Metric: {'Precision': 3.16, 'Recall': 24.66}
Func worker, run time: 4.140321969985962
Func splitData, run time: 0.9315330982208252
Experiment 9:
Metric: {'Precision': 3.17, 'Recall': 24.66}
Func worker, run time: 3.96396803855896
Average Result (M=10, N=10): {'Precision': 3.1659999999999995, 'Recall': 24.71}
Func run, run time: 51.46743392944336
alpha = 0.1
Func loadData, run time: 0.5897848606109619
Func splitData, run time: 1.2940878868103027
Experiment 0:
Metric: {'Precision': 3.16, 'Recall': 24.71}
Func worker, run time: 4.220802068710327
Func splitData, run time: 1.119277000427246
Experiment 1:
Metric: {'Precision': 3.16, 'Recall': 24.67}
Func worker, run time: 3.690171003341675
Func splitData, run time: 1.1244220733642578
Experiment 2:
Metric: {'Precision': 3.15, 'Recall': 24.52}
Func worker, run time: 3.741908073425293
Func splitData, run time: 1.0858471393585205
Experiment 3:
Metric: {'Precision': 3.15, 'Recall': 24.57}
Func worker, run time: 3.828856945037842
Func splitData, run time: 0.9331462383270264
Experiment 4:
Metric: {'Precision': 3.15, 'Recall': 24.68}
Func worker, run time: 3.770294189453125
Func splitData, run time: 0.881289005279541
Experiment 5:
Metric: {'Precision': 3.09, 'Recall': 24.13}
Func worker, run time: 3.8148388862609863
Func splitData, run time: 0.9065887928009033
Experiment 6:
Metric: {'Precision': 3.13, 'Recall': 24.52}
Func worker, run time: 3.847368001937866
Func splitData, run time: 0.8994863033294678
Experiment 7:
Metric: {'Precision': 3.09, 'Recall': 24.18}
Func worker, run time: 3.8750672340393066
Func splitData, run time: 0.9352071285247803
Experiment 8:
Metric: {'Precision': 3.14, 'Recall': 24.45}
Func worker, run time: 3.980964183807373
Func splitData, run time: 0.8821008205413818
Experiment 9:
Metric: {'Precision': 3.13, 'Recall': 24.37}
Func worker, run time: 3.780784845352173
Average Result (M=10, N=10): {'Precision': 3.135, 'Recall': 24.48}
Func run, run time: 49.395665884017944
alpha = 0.2
Func loadData, run time: 0.5564239025115967
Func splitData, run time: 1.0819451808929443
Experiment 0:
Metric: {'Precision': 3.09, 'Recall': 24.13}
Func worker, run time: 3.7844419479370117
Func splitData, run time: 1.1305460929870605
Experiment 1:
Metric: {'Precision': 3.08, 'Recall': 24.05}
Func worker, run time: 3.8170173168182373
Func splitData, run time: 0.8823709487915039
Experiment 2:
Metric: {'Precision': 3.07, 'Recall': 23.88}
Func worker, run time: 3.8645858764648438
Func splitData, run time: 0.926703929901123
Experiment 3:
Metric: {'Precision': 3.06, 'Recall': 23.85}
Func worker, run time: 3.8203530311584473
Func splitData, run time: 0.8779711723327637
Experiment 4:
Metric: {'Precision': 3.07, 'Recall': 24.07}
Func worker, run time: 4.0470640659332275
Func splitData, run time: 0.8845350742340088
Experiment 5:
Metric: {'Precision': 3.04, 'Recall': 23.76}
Func worker, run time: 3.8835208415985107
Func splitData, run time: 0.8999407291412354
Experiment 6:
Metric: {'Precision': 3.06, 'Recall': 23.94}
Func worker, run time: 3.8781838417053223
Func splitData, run time: 0.8658223152160645
Experiment 7:
Metric: {'Precision': 3.02, 'Recall': 23.59}
Func worker, run time: 3.9182791709899902
Func splitData, run time: 0.8931410312652588
Experiment 8:
Metric: {'Precision': 3.06, 'Recall': 23.88}
Func worker, run time: 3.91605281829834
Func splitData, run time: 0.9485352039337158
Experiment 9:
Metric: {'Precision': 3.05, 'Recall': 23.75}
Func worker, run time: 3.9514670372009277
Average Result (M=10, N=10): {'Precision': 3.0599999999999996, 'Recall': 23.889999999999997}
Func run, run time: 49.03358316421509
alpha = 0.3
Func loadData, run time: 0.8165087699890137
Func splitData, run time: 1.1548380851745605
Experiment 0:
Metric: {'Precision': 2.99, 'Recall': 23.33}
Func worker, run time: 3.9186530113220215
Func splitData, run time: 1.1145899295806885
Experiment 1:
Metric: {'Precision': 2.98, 'Recall': 23.23}
Func worker, run time: 3.880518913269043
Func splitData, run time: 0.9590201377868652
Experiment 2:
Metric: {'Precision': 2.98, 'Recall': 23.18}
Func worker, run time: 4.014327049255371
Func splitData, run time: 0.9428629875183105
Experiment 3:
Metric: {'Precision': 2.96, 'Recall': 23.07}
Func worker, run time: 4.099957704544067
Func splitData, run time: 0.945389986038208
Experiment 4:
Metric: {'Precision': 2.98, 'Recall': 23.37}
Func worker, run time: 3.903257131576538
Func splitData, run time: 0.8871619701385498
Experiment 5:
Metric: {'Precision': 2.95, 'Recall': 23.01}
Func worker, run time: 4.016380310058594
Func splitData, run time: 0.9477517604827881
Experiment 6:
Metric: {'Precision': 2.97, 'Recall': 23.2}
Func worker, run time: 4.101273059844971
Func splitData, run time: 0.9239673614501953
Experiment 7:
Metric: {'Precision': 2.93, 'Recall': 22.92}
Func worker, run time: 4.01638388633728
Func splitData, run time: 1.129809856414795
Experiment 8:
Metric: {'Precision': 2.97, 'Recall': 23.17}
Func worker, run time: 4.296221971511841
Func splitData, run time: 1.0508511066436768
Experiment 9:
Metric: {'Precision': 2.96, 'Recall': 23.06}
Func worker, run time: 4.31338906288147
Average Result (M=10, N=10): {'Precision': 2.9669999999999996, 'Recall': 23.154000000000003}
Func run, run time: 51.64158391952515
alpha = 0.4
Func loadData, run time: 0.5854520797729492
Func splitData, run time: 1.2064447402954102
Experiment 0:
Metric: {'Precision': 2.9, 'Recall': 22.66}
Func worker, run time: 3.82174015045166
Func splitData, run time: 1.109299898147583
Experiment 1:
Metric: {'Precision': 2.89, 'Recall': 22.59}
Func worker, run time: 3.7054030895233154
Func splitData, run time: 1.0969369411468506
Experiment 2:
Metric: {'Precision': 2.9, 'Recall': 22.58}
Func worker, run time: 3.6588618755340576
Func splitData, run time: 1.0210728645324707
Experiment 3:
Metric: {'Precision': 2.88, 'Recall': 22.49}
Func worker, run time: 4.334802150726318
Func splitData, run time: 0.9911270141601562
Experiment 4:
Metric: {'Precision': 2.89, 'Recall': 22.63}
Func worker, run time: 3.9276959896087646
Func splitData, run time: 0.8923721313476562
Experiment 5:
Metric: {'Precision': 2.88, 'Recall': 22.5}
Func worker, run time: 3.918529987335205
Func splitData, run time: 0.8772850036621094
Experiment 6:
Metric: {'Precision': 2.89, 'Recall': 22.64}
Func worker, run time: 4.415400981903076
Func splitData, run time: 0.9515841007232666
Experiment 7:
Metric: {'Precision': 2.86, 'Recall': 22.31}
Func worker, run time: 3.962172269821167
Func splitData, run time: 0.9239819049835205
Experiment 8:
Metric: {'Precision': 2.89, 'Recall': 22.53}
Func worker, run time: 4.088367938995361
Func splitData, run time: 0.9344871044158936
Experiment 9:
Metric: {'Precision': 2.89, 'Recall': 22.48}
Func worker, run time: 4.000338077545166
Average Result (M=10, N=10): {'Precision': 2.887, 'Recall': 22.540999999999997}
Func run, run time: 50.62652516365051
alpha = 0.5
Func loadData, run time: 0.6036858558654785
Func splitData, run time: 1.1832058429718018
Experiment 0:
Metric: {'Precision': 2.83, 'Recall': 22.12}
Func worker, run time: 3.846303939819336
Func splitData, run time: 1.1365282535552979
Experiment 1:
Metric: {'Precision': 2.82, 'Recall': 21.99}
Func worker, run time: 3.859900951385498
Func splitData, run time: 0.9495680332183838
Experiment 2:
Metric: {'Precision': 2.83, 'Recall': 22.04}
Func worker, run time: 4.079281806945801
Func splitData, run time: 0.9861047267913818
Experiment 3:
Metric: {'Precision': 2.83, 'Recall': 22.06}
Func worker, run time: 4.394114017486572
Func splitData, run time: 1.0009009838104248
Experiment 4:
Metric: {'Precision': 2.82, 'Recall': 22.1}
Func worker, run time: 4.027346134185791
Func splitData, run time: 0.8993139266967773
Experiment 5:
Metric: {'Precision': 2.81, 'Recall': 21.98}
Func worker, run time: 4.053209066390991
Func splitData, run time: 0.9647109508514404
Experiment 6:
Metric: {'Precision': 2.83, 'Recall': 22.13}
Func worker, run time: 4.376596927642822
Func splitData, run time: 1.0014677047729492
Experiment 7:
Metric: {'Precision': 2.78, 'Recall': 21.74}
Func worker, run time: 4.266858816146851
Func splitData, run time: 0.924774169921875
Experiment 8:
Metric: {'Precision': 2.81, 'Recall': 21.9}
Func worker, run time: 3.9737420082092285
Func splitData, run time: 0.8944978713989258
Experiment 9:
Metric: {'Precision': 2.82, 'Recall': 21.92}
Func worker, run time: 3.682781934738159
Average Result (M=10, N=10): {'Precision': 2.8180000000000005, 'Recall': 21.998}
Func run, run time: 51.33020997047424
alpha = 0.6
Func loadData, run time: 0.7814671993255615
Func splitData, run time: 1.1311962604522705
Experiment 0:
Metric: {'Precision': 2.81, 'Recall': 21.99}
Func worker, run time: 3.6779329776763916
Func splitData, run time: 1.0877008438110352
Experiment 1:
Metric: {'Precision': 2.8, 'Recall': 21.87}
Func worker, run time: 3.7011940479278564
Func splitData, run time: 0.931542158126831
Experiment 2:
Metric: {'Precision': 2.83, 'Recall': 22.02}
Func worker, run time: 4.184109926223755
Func splitData, run time: 0.9873087406158447
Experiment 3:
Metric: {'Precision': 2.81, 'Recall': 21.95}
Func worker, run time: 4.34235692024231
Func splitData, run time: 0.9976329803466797
Experiment 4:
Metric: {'Precision': 2.81, 'Recall': 21.99}
Func worker, run time: 4.107128858566284
Func splitData, run time: 1.0260448455810547
Experiment 5:
Metric: {'Precision': 2.8, 'Recall': 21.87}
Func worker, run time: 4.346803903579712
Func splitData, run time: 0.9875109195709229
Experiment 6:
Metric: {'Precision': 2.81, 'Recall': 21.97}
Func worker, run time: 4.095118045806885
Func splitData, run time: 0.954246997833252
Experiment 7:
Metric: {'Precision': 2.77, 'Recall': 21.64}
Func worker, run time: 4.133518934249878
Func splitData, run time: 0.9359958171844482
Experiment 8:
Metric: {'Precision': 2.79, 'Recall': 21.78}
Func worker, run time: 4.088289976119995
Func splitData, run time: 0.9729337692260742
Experiment 9:
Metric: {'Precision': 2.8, 'Recall': 21.83}
Func worker, run time: 4.133095979690552
Average Result (M=10, N=10): {'Precision': 2.803, 'Recall': 21.891000000000002}
Func run, run time: 51.847046852111816
alpha = 0.7
Func loadData, run time: 0.6439578533172607
Func splitData, run time: 1.2230210304260254
Experiment 0:
Metric: {'Precision': 2.8, 'Recall': 21.87}
Func worker, run time: 3.8913540840148926
Func splitData, run time: 1.1245911121368408
Experiment 1:
Metric: {'Precision': 2.79, 'Recall': 21.81}
Func worker, run time: 3.9031801223754883
Func splitData, run time: 1.241105079650879
Experiment 2:
Metric: {'Precision': 2.81, 'Recall': 21.87}
Func worker, run time: 3.9086101055145264
Func splitData, run time: 0.9072818756103516
Experiment 3:
Metric: {'Precision': 2.79, 'Recall': 21.74}
Func worker, run time: 4.126950740814209
Func splitData, run time: 0.9839358329772949
Experiment 4:
Metric: {'Precision': 2.79, 'Recall': 21.85}
Func worker, run time: 4.147830009460449
Func splitData, run time: 0.9406859874725342
Experiment 5:
Metric: {'Precision': 2.8, 'Recall': 21.84}
Func worker, run time: 4.09603214263916
Func splitData, run time: 0.9855873584747314
Experiment 6:
Metric: {'Precision': 2.79, 'Recall': 21.83}
Func worker, run time: 4.132368326187134
Func splitData, run time: 0.9442930221557617
Experiment 7:
Metric: {'Precision': 2.77, 'Recall': 21.61}
Func worker, run time: 4.14454984664917
Func splitData, run time: 0.9878599643707275
Experiment 8:
Metric: {'Precision': 2.78, 'Recall': 21.67}
Func worker, run time: 4.150234937667847
Func splitData, run time: 0.9582130908966064
Experiment 9:
Metric: {'Precision': 2.79, 'Recall': 21.75}
Func worker, run time: 4.084574937820435
Average Result (M=10, N=10): {'Precision': 2.791, 'Recall': 21.784000000000002}
Func run, run time: 51.78817391395569
alpha = 0.8
Func loadData, run time: 0.6539361476898193
Func splitData, run time: 1.23051118850708
Experiment 0:
Metric: {'Precision': 2.79, 'Recall': 21.8}
Func worker, run time: 3.9864609241485596
Func splitData, run time: 1.1422462463378906
Experiment 1:
Metric: {'Precision': 2.78, 'Recall': 21.71}
Func worker, run time: 3.915750026702881
Func splitData, run time: 0.9693622589111328
Experiment 2:
Metric: {'Precision': 2.8, 'Recall': 21.79}
Func worker, run time: 4.121142148971558
Func splitData, run time: 0.9255979061126709
Experiment 3:
Metric: {'Precision': 2.78, 'Recall': 21.69}
Func worker, run time: 4.081761598587036
Func splitData, run time: 0.9528212547302246
Experiment 4:
Metric: {'Precision': 2.78, 'Recall': 21.77}
Func worker, run time: 4.107932090759277
Func splitData, run time: 0.9039068222045898
Experiment 5:
Metric: {'Precision': 2.79, 'Recall': 21.77}
Func worker, run time: 4.103371858596802
Func splitData, run time: 0.9776840209960938
Experiment 6:
Metric: {'Precision': 2.78, 'Recall': 21.77}
Func worker, run time: 4.158017873764038
Func splitData, run time: 0.9379229545593262
Experiment 7:
Metric: {'Precision': 2.76, 'Recall': 21.54}
Func worker, run time: 4.1544189453125
Func splitData, run time: 0.9832949638366699
Experiment 8:
Metric: {'Precision': 2.76, 'Recall': 21.55}
Func worker, run time: 4.169132947921753
Func splitData, run time: 0.9408540725708008
Experiment 9:
Metric: {'Precision': 2.78, 'Recall': 21.68}
Func worker, run time: 3.8871219158172607
Average Result (M=10, N=10): {'Precision': 2.78, 'Recall': 21.707}
Func run, run time: 51.54830598831177
alpha = 0.9
Func loadData, run time: 1.0025537014007568
Func splitData, run time: 1.333798885345459
Experiment 0:
Metric: {'Precision': 2.78, 'Recall': 21.72}
Func worker, run time: 4.1268157958984375
Func splitData, run time: 1.2158279418945312
Experiment 1:
Metric: {'Precision': 2.77, 'Recall': 21.64}
Func worker, run time: 4.124691963195801
Func splitData, run time: 0.9392058849334717
Experiment 2:
Metric: {'Precision': 2.79, 'Recall': 21.73}
Func worker, run time: 4.153153896331787
Func splitData, run time: 1.040349006652832
Experiment 3:
Metric: {'Precision': 2.78, 'Recall': 21.65}
Func worker, run time: 4.2734057903289795
Func splitData, run time: 0.9350490570068359
Experiment 4:
Metric: {'Precision': 2.77, 'Recall': 21.74}
Func worker, run time: 4.1106908321380615
Func splitData, run time: 0.9704811573028564
Experiment 5:
Metric: {'Precision': 2.78, 'Recall': 21.74}
Func worker, run time: 4.073996067047119
Func splitData, run time: 0.9093971252441406
Experiment 6:
Metric: {'Precision': 2.78, 'Recall': 21.71}
Func worker, run time: 4.173330783843994
Func splitData, run time: 0.950319766998291
Experiment 7:
Metric: {'Precision': 2.75, 'Recall': 21.5}
Func worker, run time: 4.211034059524536
Func splitData, run time: 0.9746339321136475
Experiment 8:
Metric: {'Precision': 2.76, 'Recall': 21.51}
Func worker, run time: 4.386167049407959
Func splitData, run time: 1.059217929840088
Experiment 9:
Metric: {'Precision': 2.78, 'Recall': 21.63}
Func worker, run time: 4.3950629234313965
Average Result (M=10, N=10): {'Precision': 2.774, 'Recall': 21.657}
Func run, run time: 53.61407208442688
alpha = 1.0
Func loadData, run time: 0.759735107421875
Func splitData, run time: 1.224877119064331
Experiment 0:
Metric: {'Precision': 1.82, 'Recall': 14.18}
Func worker, run time: 3.729387044906616
Func splitData, run time: 1.0924749374389648
Experiment 1:
Metric: {'Precision': 1.8, 'Recall': 14.09}
Func worker, run time: 3.4529192447662354
Func splitData, run time: 1.1971077919006348
Experiment 2:
Metric: {'Precision': 1.82, 'Recall': 14.17}
Func worker, run time: 3.3798270225524902
Func splitData, run time: 0.9837942123413086
Experiment 3:
Metric: {'Precision': 1.79, 'Recall': 13.97}
Func worker, run time: 3.9176063537597656
Func splitData, run time: 0.9464359283447266
Experiment 4:
Metric: {'Precision': 1.8, 'Recall': 14.13}
Func worker, run time: 3.7294299602508545
Func splitData, run time: 0.9589240550994873
Experiment 5:
Metric: {'Precision': 1.81, 'Recall': 14.16}
Func worker, run time: 3.68717098236084
Func splitData, run time: 0.9121508598327637
Experiment 6:
Metric: {'Precision': 1.82, 'Recall': 14.22}
Func worker, run time: 3.720194101333618
Func splitData, run time: 0.9417569637298584
Experiment 7:
Metric: {'Precision': 1.8, 'Recall': 14.05}
Func worker, run time: 3.7353739738464355
Func splitData, run time: 0.9648470878601074
Experiment 8:
Metric: {'Precision': 1.78, 'Recall': 13.9}
Func worker, run time: 3.7211191654205322
Func splitData, run time: 0.9093520641326904
Experiment 9:
Metric: {'Precision': 1.81, 'Recall': 14.08}
Func worker, run time: 3.725862979888916
Average Result (M=10, N=10): {'Precision': 1.8050000000000002, 'Recall': 14.094999999999999}
Func run, run time: 47.9080491065979